In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
train = pd.read_csv('data/train.csv', parse_dates = True)
stores = pd.read_csv('data/store.csv')

/opt/anaconda3/envs/minicomp/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
fillna_storetype_a = stores.groupby('StoreType').median().loc['a', 'CompetitionDistance']
fillna_storetype_b = stores.groupby('StoreType').median().loc['b', 'CompetitionDistance']
fillna_storetype_c = stores.groupby('StoreType').median().loc['c', 'CompetitionDistance']
fillna_storetype_d = stores.groupby('StoreType').median().loc['d', 'CompetitionDistance']
stores.loc[(stores['CompetitionDistance'].isnull()) & (stores.loc[:,'StoreType'] == 'a')] = fillna_storetype_a
stores.loc[(stores['CompetitionDistance'].isnull()) & (stores.loc[:,'StoreType'] == 'b')] = fillna_storetype_b
stores.loc[(stores['CompetitionDistance'].isnull()) & (stores.loc[:,'StoreType'] == 'c')] = fillna_storetype_c
stores.loc[(stores['CompetitionDistance'].isnull()) & (stores.loc[:,'StoreType'] == 'd')] = fillna_storetype_d

train.loc[(train.loc[:, 'Sales'].isnull()) & (train['Open'] == 0), 'Sales'] = 0
train.loc[(train.loc[:, 'Sales'].isnull()) & (train['Customers'] == 0), 'Sales'] = 0

train.loc[(train.loc[:, 'Customers'].isnull()) & (train['Sales'] == 0), 'Customers'] = 0

no_null_sales = train.loc[(train['Sales'].notnull()) | (train['Open'] == 0), :]
no_null_sales.loc[:,'Sales'].fillna(0, inplace=True)

no_null_sales = no_null_sales.loc[no_null_sales['Store'].notnull(), :]

no_null_sales.loc[:,'Date'] = pd.to_datetime(no_null_sales.loc[:,'Date'], format='%Y-%m-%d')
no_null_sales.loc[:,'DayOfWeek'] = no_null_sales.loc[:,'Date'].dt.weekday + 1

no_null_sales.loc[no_null_sales['StateHoliday'] == 0.0, 'StateHoliday'] = '0'
no_null_sales.loc[no_null_sales['StateHoliday'] == 0, 'StateHoliday'] = '0'

no_null_sales.loc[(no_null_sales['Open'].isnull()) & (no_null_sales['Sales'] == 0),'Open'] = 0
no_null_sales.loc[(no_null_sales['Open'].isnull()) & (no_null_sales['Sales'] > 0),'Open'] = 1

/opt/anaconda3/envs/minicomp/lib/python3.8/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [4]:
#WILL TAKE 30 MINS TO RUN!!!!!!
#Slow and ugly way to fill in NaNs with the mode
holiday_nulls = no_null_sales.loc[no_null_sales['StateHoliday'].isnull()]
for i, row in holiday_nulls.iterrows():
    date = str(row['Date'])
    mode = no_null_sales.loc[no_null_sales['Date'] == date, 'StateHoliday'].mode()[0]
    no_null_sales.loc[i, 'StateHoliday'] = mode
    
holiday_nulls = no_null_sales.loc[no_null_sales['SchoolHoliday'].isnull()]
for i, row in holiday_nulls.iterrows():
    date = str(row['Date'])
    mode = no_null_sales.loc[no_null_sales['Date'] == date, 'SchoolHoliday'].mode()[0]
    no_null_sales.loc[i, 'SchoolHoliday'] = mode

KeyboardInterrupt: 

In [ ]:
customer_means = no_null_sales.groupby(['Store', 'DayOfWeek']).mean()['Customers']

customer_nulls = no_null_sales.loc[no_null_sales['Customers'].isnull()]
for i, row in customer_nulls.iterrows():
    store = row['Store']
    DayOfWeek = row['DayOfWeek']
    mean = customer_means.loc[store, DayOfWeek]
    no_null_sales.loc[i, 'Customers'] = mean

In [ ]:
no_null_sales.reset_index(inplace = True, drop = True)

null_promo = no_null_sales.loc[no_null_sales['Promo'].isnull()]

for i, row in null_promo.iterrows():
    above_and_below = no_null_sales.loc[i-1, 'Promo'] + no_null_sales.loc[i+1, 'Promo']
    if above_and_below == 2:
        no_null_sales.loc[i, 'Promo'] = 1
    elif above_and_below == 0:
        no_null_sales.loc[i, 'Promo'] = 0
        
null_promo = no_null_sales.loc[no_null_sales['Promo'].isnull()]

for i, row in null_promo.iterrows():
    date = str(row['Date'])
    mean = no_null_sales.groupby('Date').mean().loc[date, 'Promo']
    if mean > 0.9:
        no_null_sales.loc[i, 'Promo'] = 1
    else:
        no_null_sales.loc[i, 'Promo'] = 0

In [ ]:
no_null_sales.to_csv('super_clean_data.csv', index = False)